In [6]:
def log_transaction(func):
    def anywey():
        print('1 Log de la transacción...')
        func()
        print('3 Log terminado...')
    return anywey


@log_transaction
def process_payment():
    print('2 Procesando pago....')

process_payment()

1 Log de la transacción...
2 Procesando pago....
3 Log terminado...


In [ ]:
def check_access(func):
    def wrapper(employee):
        # Comprobar si el empleado tiene rol 'admin'
        if employee.get('role') == 'admin':
            return func(employee)
        else:
            print('ACCESO DENEGADO. Solo los administradores pueden acceder.')
    return wrapper

@check_access
def delete_employee(employee):
    print(f'Elempleado {employee['name']} ha sido eliminado.')

admin = {'name': 'Carlos', 'role': 'admin'}
employee = {'name': 'Ana', 'role': 'employee'}


#delete_employee(admin)
delete_employee(employee)

In [17]:
# Decorador que comprueba si un empleado tiene un rol especifico

def check_accesos(requerid_role):
    def decorator(func):
        def wrapper(employee):
            # Comprobar si el rol del empleado coincide con el rol requerido
            if employee.get('role') == requerid_role:
                return func(employee)
            else:
                print(f"ACCESO DENEGADO. Solo {requerid_role} pueden realizar esta acción")
        return wrapper
    return decorator

def log_action(func):
    def wrapper(employee):
        print(f"Registrando acción para el empleado {employee['name']}")
        return func(employee)
    return wrapper


@check_accesos('user')
@log_action
def delete_employee(employee):
    print(f"El empleado {employee['name']} ha sido eliminado.")

admin = {'name': 'Carlos', 'role': 'admin'}
user = {'name': 'Paco', 'role': 'user'}

#delete_employee(admin)
delete_employee(user)

Registrando acción para el empleado Paco
El empleado Paco ha sido eliminado.


In [18]:
import pandas as pd
import time

# Decorador 1: Valida que el DataFrame no esté vacío
def validar_dataframe(func):
    # 'func' es la función que vamos a decorar (ej. preprocesar_datos).
    def wrapper(df, *args, **kwargs):
        # 'wrapper' es la nueva función que reemplaza a la original.
        # Recibe un DataFrame 'df' y cualquier otro argumento.
        print("-> [Validación]: Comprobando si el DataFrame tiene datos...")
        if df.empty:
            # Si el DataFrame está vacío, no ejecutes la función original.
            print("   ADVERTENCIA: El DataFrame está vacío. Se detiene la ejecución.")
            return None # Detenemos todo y devolvemos None.

        # Si no está vacío, permite que la función original se ejecute.
        print("   [Validación]: OK. El DataFrame tiene datos.")
        return func(df, *args, **kwargs)
    return wrapper # El decorador devuelve la función 'wrapper'.

# Decorador 2: Mide el tiempo de ejecución de una función
def medir_tiempo(func):
    # 'func' es la función que vamos a decorar.
    def wrapper(*args, **kwargs):
        # 'wrapper' recibe cualquier tipo de argumento.
        print("-> [Medición]: Iniciando cronómetro...")
        start_time = time.time() # Guarda el tiempo de inicio.

        result = func(*args, **kwargs) # Ejecuta la función original y guarda su resultado.

        end_time = time.time() # Guarda el tiempo de finalización.
        print(f"   [Medición]: OK. La función tardó {end_time - start_time:.4f} segundos.")
        return result # Devuelve el resultado original de la función.
    return wrapper # El decorador devuelve la función 'wrapper'.

@validar_dataframe
@medir_tiempo
def preprocesar_datos(df: pd.DataFrame) -> pd.DataFrame:
    """
    Función que simula una limpieza de datos en un DataFrame.
    """
    print("--> [Procesamiento]: Realizando limpieza de datos...")
    # Simula un trabajo que toma tiempo
    time.sleep(1)
    df_procesado = df.dropna()
    print("--> [Procesamiento]: ¡Limpieza completada!")
    return df_procesado

# --- Caso 1: DataFrame con datos ---
print("--- INICIANDO PRUEBA CON DATOS VÁLIDOS ---")
datos_validos = pd.DataFrame({'col1': [1, 2, 3], 'col2': [4, 5, None]})
preprocesar_datos(datos_validos)

print("\n" + "="*50 + "\n")

# --- Caso 2: DataFrame vacío ---
print("--- INICIANDO PRUEBA CON DATOS VACÍOS ---")
datos_vacios = pd.DataFrame()
preprocesar_datos(datos_vacios)

--- INICIANDO PRUEBA CON DATOS VÁLIDOS ---
-> [Validación]: Comprobando si el DataFrame tiene datos...
   [Validación]: OK. El DataFrame tiene datos.
-> [Medición]: Iniciando cronómetro...
--> [Procesamiento]: Realizando limpieza de datos...
--> [Procesamiento]: ¡Limpieza completada!
   [Medición]: OK. La función tardó 1.0702 segundos.


--- INICIANDO PRUEBA CON DATOS VACÍOS ---
-> [Validación]: Comprobando si el DataFrame tiene datos...
   ADVERTENCIA: El DataFrame está vacío. Se detiene la ejecución.


# Decoradores: "Superpoderes" para tus Funciones

Los **decoradores** son una forma de "envolver" una función para añadirle una nueva funcionalidad antes o después de que se ejecute, **sin modificar su código interno**.

**La Analogía:**
Piensa en un **regalo**.
* La **función original** es el **regalo** en sí (ej. una función que procesa un pago).
* Un **decorador** es el **papel de regalo y el moño**. No cambia el regalo, pero añade algo extra por fuera (ej. registrar la hora de la transacción, verificar permisos).

Se usan con el símbolo `@` justo encima de la definición de la función.

## 1. La Anatomía de un Decorador Simple

Un decorador es una función que recibe otra función como argumento, le añade funcionalidades y devuelve una nueva función "mejorada".

In [19]:
# Este es nuestro decorador. Recibe una función (func) como parámetro.
def log_decorator(func):
    # 'wrapper' es la nueva función que "envuelve" a la original.
    def wrapper():
        print(f"Iniciando log para la función: {func.__name__}...")
        func() # Aquí se ejecuta la función original.
        print(f"Log para la función {func.__name__} terminado.")
    # El decorador devuelve la función 'wrapper'.
    return wrapper

# Aplicamos el decorador a nuestra función
@log_decorator
def procesar_pago():
    print("...Procesando pago...")

# Al llamar a procesar_pago(), en realidad estamos llamando a la versión "envuelta" (wrapper).
procesar_pago()

Iniciando log para la función: procesar_pago...
...Procesando pago...
Log para la función procesar_pago terminado.


## 2. Decoradores que Aceptan Argumentos

A veces, queremos que nuestro decorador sea configurable. Por ejemplo, un decorador que verifique permisos necesita saber qué permiso verificar. Para esto, añadimos una capa extra de anidación.

In [20]:
# 1. La función más externa recibe el argumento del decorador.
def verificar_acceso(rol_requerido):
    # 2. Esta función es el decorador real.
    def decorator(func):
        # 3. El wrapper es el que finalmente se ejecuta.
        def wrapper(empleado):
            print(f"Verificando si el empleado tiene el rol '{rol_requerido}'...")
            if empleado.get('rol') == rol_requerido:
                return func(empleado) # Si tiene permiso, ejecuta la función original.
            else:
                print(f"Acceso denegado: Se requiere el rol de '{rol_requerido}'.")
        return wrapper
    return decorator

# Aplicamos el decorador pasándole un argumento.
@verificar_acceso('admin')
def eliminar_empleado(empleado):
    print(f"Éxito: El empleado {empleado['nombre']} ha sido eliminado.")

admin = {'nombre': 'Carlos', 'rol': 'admin'}
usuario = {'nombre': 'Ana', 'rol': 'usuario'}

eliminar_empleado(admin)
print("-" * 20)
eliminar_empleado(usuario)

Verificando si el empleado tiene el rol 'admin'...
Éxito: El empleado Carlos ha sido eliminado.
--------------------
Verificando si el empleado tiene el rol 'admin'...
Acceso denegado: Se requiere el rol de 'admin'.


## 3. Decoradores Anidados: Apilando Comportamientos

Puedes aplicar múltiples decoradores a una misma función. Se ejecutan en orden desde el que está más abajo (más cerca de la función) hacia arriba.

In [ ]:
# Aplicamos ambos decoradores que ya creamos.
@verificar_acceso('admin') # 2º en ejecutarse
@log_decorator           # 1º en ejecutarse (el más cercano a la función)
def tarea_sensible():
    print("...Realizando tarea sensible...")

# En este caso, primero se ejecutará el log y luego la verificación de acceso.
tarea_sensible() # Esto fallará porque no le pasamos el diccionario de empleado

## Ejercicio Práctico para Ciencia de Datos: Medidor de Tiempo y Validador de Datos

Vamos a crear dos decoradores muy útiles en el día a día de un científico de datos:
1.  **`@timer`**: Medirá y mostrará cuánto tiempo tarda en ejecutarse una función.
2.  **`@validate_input`**: Se asegurará de que la entrada a nuestra función de procesamiento sea del tipo correcto.

In [ ]:
import time
import pandas as pd

# --- DECORADOR 1: Medidor de Tiempo ---
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time() # 1. Guarda la hora de inicio
        result = func(*args, **kwargs) # 2. Ejecuta la función
        end_time = time.time() # 3. Guarda la hora de fin
        print(f"La función '{func.__name__}' tardó {end_time - start_time:.4f} segundos en ejecutarse.")
        return result
    return wrapper

# --- DECORADOR 2: Validador de Entrada ---
def validate_input(func):
    def wrapper(data):
        # Verifica si la entrada es un DataFrame de Pandas
        if not isinstance(data, pd.DataFrame):
            # Lanza un error si el tipo no es correcto
            raise TypeError("La entrada debe ser un DataFrame de Pandas.")

        # Verifica si el DataFrame está vacío
        if data.empty:
            print("Advertencia: El DataFrame de entrada está vacío. No se procesará nada.")
            return None # Termina la ejecución si está vacío

        # Si todo está bien, ejecuta la función original
        return func(data)
    return wrapper


# --- APLICANDO LOS DECORADORES A UNA FUNCIÓN DE PROCESAMIENTO ---
@timer
@validate_input
def procesar_datos(df):
    """Simula un proceso de datos pesado."""
    print(f"Procesando un DataFrame con {len(df)} filas...")
    # Simulamos un trabajo largo
    time.sleep(2)
    print("Proceso terminado.")
    return "Resultados del procesamiento"

# --- PROBANDO NUESTRA FUNCIÓN DECORADA ---
# 1. Prueba con datos correctos
df_bueno = pd.DataFrame({'col1': [1, 2, 3]})
procesar_datos(df_bueno)

print("\n" + "="*30 + "\n")

# 2. Prueba con un DataFrame vacío
df_vacio = pd.DataFrame()
procesar_datos(df_vacio)

print("\n" + "="*30 + "\n")

# 3. Prueba con un tipo de dato incorrecto
lista_mala = [1, 2, 3]
try:
    procesar_datos(lista_mala)
except TypeError as e:
    print(f"Error capturado: {e}")